**Plot Marmousi Snapshots**

**Daniel Köhn**

**Kiel, 24/06/2016**

**Import necessary packages**

In [ ]:
from __future__ import division
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from matplotlib.colors import LightSource, Normalize
from matplotlib.pyplot import gca
from pylab import rcParams
from matplotlib import rc
import scipy.ndimage.filters
import pickle

**FD grid dimensions**

In [ ]:
DH = 20.0
NX = 500
NY = 174
N = NX*NY;

**Wavefield snapshot parameters**

In [ ]:
clip_div = 1.      # div wavefield clipping
clip_rot = 1.      # rot wavefield clipping
NSNAP1 = 1         # first snapshot
NSNAP2 = 50        # last snapshot
DSNAP = 1          # snapshot increment
TSNAP1 = 0.002     # time of first snapshot
TSNAP2 = 3.0       # time of last snapshot
TSNAPINC = 0.06    # time increment between snapshots

**Define fonts**

In [ ]:
FSize = 25
font = {'color':  'black',
        'weight': 'bold',
        'size': FSize}
mpl.rc('xtick', labelsize=FSize) 
mpl.rc('ytick', labelsize=FSize) 
rcParams['figure.figsize'] = 20, 8

**Read S-wave velocity model and RTM result**

In [ ]:
f = open ('start/marmousi_II_marine.vp')
data_type = np.dtype ('float32').newbyteorder ('<')
vp = np.fromfile (f, dtype=data_type)
vp = vp.reshape(NX,NY)
vp = np.transpose(vp)
vp = np.flipud(vp)

In [ ]:
def load_snap(offset):
    
    f1 = open ('snap/waveform_forward.bin.div')
    f2 = open ('snap/waveform_forward.bin.rot')
    
    data_type1 = np.dtype ('float32').newbyteorder ('<')
    offset_snap = 4*NX*NY*(offset-1)
    
    snap = np.memmap(f1, dtype=data_type1, mode='c', shape=(NX,NY), offset=offset_snap)
    snap = np.transpose(snap)
    snap = np.flipud(snap)
    
    snap1 = np.memmap(f2, dtype=data_type1, mode='c', shape=(NX,NY), offset=offset_snap)
    snap1 = np.transpose(snap1)
    snap1 = np.flipud(snap1)
    
    return snap, snap1

**Define Axis**

In [ ]:
x = np.arange(DH, DH*(NX+1), DH)
y = np.arange(DH, DH*(NY+1), DH)

# convert from [m] -> [km]
x = x / 1000.
y = y / 1000.

**Plot $\alpha$-Blending of Vp model (Gray) and snapshots (Seismic)**

In [ ]:
extent = [np.min(x),np.max(x),np.min(y),np.max(y)]

fig = plt.figure(frameon=True)


#plt.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
#plt.rc('text', usetex=True)

i = NSNAP1
while i <= NSNAP2:
    
    offset = i;
    time = TSNAP1 + (i-1) * TSNAPINC
    
    # import snapshot with offset
    snap, snap1 = load_snap(offset);

    ax=plt.subplot(2, 1, 1)
    
    # plot vp model
    im1 = plt.imshow(vp, cmap=plt.cm.gray, interpolation='nearest', extent=extent)

    # superimpose semi-transparent div-snapshot on vp model
    im2 = plt.imshow(snap, cmap=plt.cm.seismic, alpha=.75, interpolation='bicubic',
                     extent=extent, vmin=-clip_div, vmax=clip_div)

    # add title and axis annotations
    #a = gca()
    #a.set_xticklabels(a.get_xticks(), font)
    #a.set_yticklabels(a.get_yticks(), font)
    time_str = str(time)
    name_title = "P-wave energy (time = " + time_str[0:5] + " s)"
    plt.title(name_title, fontdict=font)
    plt.ylabel('Depth [km]', fontdict=font)
    plt.gca().invert_yaxis()
    plt.tight_layout()
    
    ax=plt.subplot(2, 1, 2)
    
    # plot vp model
    im1 = plt.imshow(vp, cmap=plt.cm.gray, interpolation='nearest', extent=extent)

    # superimpose semi-transparent div-snapshot on vp model
    im2 = plt.imshow(snap1, cmap=plt.cm.seismic, alpha=.75, interpolation='bicubic',
                     extent=extent, vmin=-clip_rot, vmax=clip_rot)

    # add title and axis annotations
    #a = gca()
    #a.set_xticklabels(a.get_xticks(), font)
    #a.set_yticklabels(a.get_yticks(), font)
    name_title = "S-wave energy"
    plt.title(name_title, fontdict=font)
    plt.ylabel('Depth [km]', fontdict=font)
    plt.xlabel('Distance [km]', fontdict=font)
    plt.gca().invert_yaxis()
    plt.tight_layout()
    
    # export snapshots
    #name_snap = "pics/Marmousi_snap_" + "%0.*f" %(0,np.fix(offset)) + ".pdf"
    #plt.savefig(name_snap, bbox_inches='tight', format='pdf')
    
    name_snap = "pics/Marmousi_snap_" + "%0.*f" %(0,np.fix(offset)) + ".png"
    plt.savefig(name_snap, bbox_inches='tight', format='png', dpi=100)
    
    i = i + DSNAP;